### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.5 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 0.75 # percentage point specifying the validation set end point (1.0 means no test set)
system.balance_data = 1
system.scale_data = 1

### ------------------------------------------------------------------------------------------------------------

### Base test

In [ ]:
feature_name = 'day'

In [ ]:
# Train ensemble on train data
clf, scaler = train_clf_ensemble(LogisticRegression, data, ensemble_size=10)

In [ ]:
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### Search for best day of week to trade

In [ ]:
# Compute the profit factor for every candidate value
levels = [0,1,2,3,4]
pfs = []
nts = []
for l in tqdm(levels):
    pf, ntrades = compute_stats(data, filter_trades_by_feature(trades, data, featformat(feature_name), exact_value=l))
    pfs.append(pf)
    nts.append(len(ntrades))

In [ ]:
# Plot the optimization/search results
plt.plot(levels, pfs);
plt.xlabel(feature_name);
plt.ylabel('Profit Factor');

In [ ]:
res = pd.DataFrame(data = np.hstack([ np.array(nts).reshape(-1,1),
                                      np.array(pfs).reshape(-1,1)]),
             index=np.array(levels),
             columns=['num trades', 'profit factor'])
res

In [ ]:
best_day = 4

### Test on unseen data

In [ ]:
# Base test without the filter
equity, pf, trades = qbacktest(clf, scaler, data, skip_val=1, skip_test=0)

In [ ]:
# Test with the filter
equity, pf, trades = qbacktest(clf, scaler, data, quiet=1, skip_val=1, skip_test=0)
# filter stats
pf, ntrades = compute_stats(data, filter_trades_by_feature(trades, data, featformat(feature_name), exact_value=best_day))
print(f'Profit factor: {get_profit_factor(ntrades):.5f}, Winners: {get_winner_pct(ntrades):.2f}%, Trades: {len(ntrades)}')

In [ ]:
plot(ntrades['profit'].cumsum());

In [ ]:
ntrades[0:20]

### ------------------------------------------------------------------------------------------------------------